# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В датафрейме ни в обном из столбцов нет пропусков.

In [4]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

## Разобьем данные на выборки

In [5]:
train_valid, test = train_test_split( df, test_size=0.25, random_state=12345)
df_train, df_valid = train_test_split(train_valid, test_size=0.3, random_state=12345)

## Исследуем модели

Сохраним признаки и целевой признак в следующих переменных для обучающей, валидационной и тестовой выборок

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

Рассмотрим 2 метода построения моделей классификации: DecisionTreeClassifier и RandomForestClassifier

### DecisionTreeClassifier

In [7]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print("Accuracy:", result)

Accuracy: 0.7482710926694329


Попробуем улучшить модель и добавим параметр глубины дерева max_depth

In [8]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions =  model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model
        best_result = result   
print('max_depth =', depth, ": ", end='')
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

max_depth = 10 : Accuracy наилучшей модели на валидационной выборке: 0.8022130013831259


Значение Accuracy стало выше и при max_depth = 10 является максимальным. 

### RandomForestClassifier

In [9]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = accuracy_score(target_valid, predictions)
print("Accuracy:", result)

Accuracy: 0.8188105117565698


Проверим как модель реагирует на изменение гиперпараметра n_estimators

In [10]:
best_model = None
best_result = 0
for est in range(1, 100, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result.max()
    print('n_estimators =', est, ": ", end='')
    print(best_result)

n_estimators = 1 : 0.7524204702627939
n_estimators = 11 : 0.8008298755186722
n_estimators = 21 : 0.8188105117565698
n_estimators = 31 : 0.8188105117565698
n_estimators = 41 : 0.8257261410788381
n_estimators = 51 : 0.8257261410788381
n_estimators = 61 : 0.8257261410788381
n_estimators = 71 : 0.8257261410788381
n_estimators = 81 : 0.8257261410788381
n_estimators = 91 : 0.8257261410788381


Видим, что модель улучшается при увеличении значения n_estimators, притом достигает максимальное значение на пятом десятке и далее не увеличивается, поэтому берем лучшее значение n_estimators = 50

Проверим как улучшается модель при разных значениях гиперпараметра max_depth глубина дерева

In [11]:
for depth in range(1, 101, 10):
    model = RandomForestClassifier(random_state = 12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print('max_depth =', depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7828492392807745
max_depth = 11 : 0.8257261410788381
max_depth = 21 : 0.8312586445366529
max_depth = 31 : 0.8188105117565698
max_depth = 41 : 0.8188105117565698
max_depth = 51 : 0.8188105117565698
max_depth = 61 : 0.8188105117565698
max_depth = 71 : 0.8188105117565698
max_depth = 81 : 0.8188105117565698
max_depth = 91 : 0.8188105117565698


In [12]:
for depth in range(20, 31):
    model = RandomForestClassifier(random_state = 12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print('max_depth =', depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 20 : 0.8188105117565698
max_depth = 21 : 0.8312586445366529
max_depth = 22 : 0.8201936376210235
max_depth = 23 : 0.8174273858921162
max_depth = 24 : 0.8174273858921162
max_depth = 25 : 0.8188105117565698
max_depth = 26 : 0.8188105117565698
max_depth = 27 : 0.8188105117565698
max_depth = 28 : 0.8188105117565698
max_depth = 29 : 0.8201936376210235
max_depth = 30 : 0.8215767634854771


Итак, при значении max_depth=21 модель лучше всего и accuracy = 0.83

## Проверка модели на тестовой выборке

### DecisionTreeClassifier

Проверим модель RandomForestClassifier на тестовой выборке

In [13]:
test_predictions = model.predict(features_test)

def accuracy(answers, predictions):
    correct = 0
    for i in range(len(answers)):
        if answers[i] == predictions[i]:
            correct += 1# < напишите код здесь >
    return correct / len(answers) 
print("Accuracy:", accuracy(target_test.values, test_predictions))

Accuracy: 0.7935323383084577


При тестировании модели получилось Accuracy = 0.79, что меньше, чем на тренировочных данных.